In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coihaique,CL,2021-10-12 21:51:27,-45.5752,-72.0662,57.81,50,40,6.91,scattered clouds
1,1,Ambilobe,MG,2021-10-12 21:41:48,-13.2000,49.0500,75.18,77,100,1.45,overcast clouds
2,2,Ushuaia,AR,2021-10-12 21:51:27,-54.8000,-68.3000,55.06,38,75,18.41,broken clouds
3,3,Bara,NG,2021-10-12 21:41:49,10.3744,10.7288,78.78,43,72,3.83,broken clouds
4,4,Christchurch,NZ,2021-10-12 21:40:47,-43.5333,172.6333,45.00,89,75,11.01,moderate rain


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 90


In [9]:

# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Ambilobe,MG,2021-10-12 21:41:48,-13.2000,49.0500,75.18,77,100,1.45,overcast clouds
3,3,Bara,NG,2021-10-12 21:41:49,10.3744,10.7288,78.78,43,72,3.83,broken clouds
13,13,Puerto Ayora,EC,2021-10-12 21:51:30,-0.7393,-90.3518,80.56,87,58,8.01,broken clouds
20,20,Kedougou,SN,2021-10-12 21:41:52,12.5579,-12.1743,77.58,94,40,9.22,light rain
21,21,Butaritari,KI,2021-10-12 21:51:32,3.0707,172.7902,81.57,68,41,7.16,scattered clouds
23,23,Avarua,CK,2021-10-12 21:51:33,-21.2078,-159.7750,77.05,69,75,11.50,light rain
27,27,Ternate,ID,2021-10-12 21:41:53,0.8000,127.4000,81.50,80,99,2.95,overcast clouds
30,30,Isangel,VU,2021-10-12 21:51:35,-19.5500,169.2667,82.51,69,75,9.22,light rain
32,32,Kapaa,US,2021-10-12 21:51:35,22.0752,-159.3190,80.92,78,75,4.00,moderate rain
39,39,Catamarca,AR,2021-10-12 21:50:53,-28.4696,-65.7852,89.78,48,35,11.01,scattered clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Ambilobe,MG,2021-10-12 21:41:48,-13.2000,49.0500,75.18,77,100,1.45,overcast clouds
3,3,Bara,NG,2021-10-12 21:41:49,10.3744,10.7288,78.78,43,72,3.83,broken clouds
13,13,Puerto Ayora,EC,2021-10-12 21:51:30,-0.7393,-90.3518,80.56,87,58,8.01,broken clouds
20,20,Kedougou,SN,2021-10-12 21:41:52,12.5579,-12.1743,77.58,94,40,9.22,light rain
21,21,Butaritari,KI,2021-10-12 21:51:32,3.0707,172.7902,81.57,68,41,7.16,scattered clouds
...,...,...,...,...,...,...,...,...,...,...,...
546,546,Carutapera,BR,2021-10-12 21:44:31,-1.1950,-46.0200,80.42,73,6,11.74,clear sky
550,550,Bacolod,PH,2021-10-12 21:40:15,10.6667,122.9500,78.35,88,99,9.08,overcast clouds
552,552,Valdosta,US,2021-10-12 21:44:33,30.8327,-83.2785,84.11,62,1,8.05,clear sky
555,555,Cururupu,BR,2021-10-12 21:44:34,-1.8283,-44.8683,78.55,77,7,5.66,clear sky


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ambilobe,MG,75.18,overcast clouds,-13.2000,49.0500,
3,Bara,NG,78.78,broken clouds,10.3744,10.7288,
13,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
20,Kedougou,SN,77.58,light rain,12.5579,-12.1743,
21,Butaritari,KI,81.57,scattered clouds,3.0707,172.7902,
23,Avarua,CK,77.05,light rain,-21.2078,-159.7750,
27,Ternate,ID,81.50,overcast clouds,0.8000,127.4000,
30,Isangel,VU,82.51,light rain,-19.5500,169.2667,
32,Kapaa,US,80.92,moderate rain,22.0752,-159.3190,
39,Catamarca,AR,89.78,scattered clouds,-28.4696,-65.7852,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ambilobe,MG,75.18,overcast clouds,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
13,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,Kedougou,SN,77.58,light rain,12.5579,-12.1743,Relais de Kédougou
21,Butaritari,KI,81.57,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
23,Avarua,CK,77.05,light rain,-21.2078,-159.7750,Paradise Inn


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))